# Brainstorm

- Task 1: Load search filter from https://docs.google.com/spreadsheets/d/1YvmsBmfjYxYi-aYJE2EX0566vyhxqMTwdir8zUtK-ls/edit#gid=2030777624

- Task 2: Search for the lessons

- Task 3: Get the from QLink to .csv
    - Task 3.1: Go through each page and copy paste the content to a doc, stored as dictionary
    - Task 3.2: Click PDF, download PDF, load PDF content to the dictionary
    
- Task 4: Load the content to Grammarly
    - Task 4.1:
    
- Task 5: Report the content to a csv
    - Task 5.1: Click plagiarism
    - Task 5.1: Identify text 

# Task 0: Standard Import, Initialize Browser

In [192]:
# Import libraries and packages for the project 
print('- Importing packages ...')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd
import numpy as np
import os
import csv
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages')
import PyPDF2
from tika import parser

import gspread
import gspread_dataframe as gd
import time
import os
from oauth2client.service_account import ServiceAccountCredentials
print('- Finish importing packages.')

# setup selenium webdriver
opt = webdriver.ChromeOptions()
#opt.add_extension("Block-image_v1.1.crx")
opt.add_argument('--disable-gpu')
opt.add_argument("--window-size=1920,1080")
opt.add_argument("--start-maximized")
opt.add_argument('--disable-dev-shm-usage')
opt.add_argument('--no-sandbox')
opt.add_argument('--ignore-certificate-errors')
opt.add_argument('--disable-notifications')
opt.add_argument("--disable-popup-blocking")
#opt.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36")
print('- Opening a web browser ...')
driver = webdriver.Chrome(options=opt)

# setup wait
wait = WebDriverWait(driver, 20)
sleep(2)

- Importing packages ...
- Finish importing packages.
- Opening a web browser ...


In [193]:
# https://www.grammarly.com/signin
# https://link.quipper.com/

print('- Accessing https://link.quipper.com/ ...')
url = 'https://link.quipper.com/ '# input('Please input URL to the text box (https://link.quipper.com/): ')
driver.get(url)
print('- https://link.quipper.com/ accessed!')
sleep(2)

- Accessing https://link.quipper.com/ ...
- https://link.quipper.com/ accessed!


# Task 1: Access the URL, Login

In [194]:
# Task 1.2: Key in login credentials
print('- Logging ...')
email_field = driver.find_element_by_name('email_or_username')
email_field.clear()
email = input('Please input your email: ') # daphne.huynh@quipper.com
email_field.send_keys(email)
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element_by_xpath('//*[@id="mount"]/div[1]/div[2]/div/div[2]/form/div[2]/div[1]/div[2]/div/input')
password_field.clear()
password = input('Please input your password: ') # Daphne@Quipper123
password_field.send_keys(password)
print('- Finish keying in password')
sleep(3)

password_field = driver.find_element_by_xpath('//*[@id="mount"]/div[1]/div[2]/div/div[2]/form/div[2]/div[1]/div[2]/div/input')
password_field.send_keys(Keys.RETURN)
print('- Successful Login!')
sleep(3)

- Logging ...
- Finish keying in email
- Finish keying in password
- Successful Login!


In [195]:
# Select School

print('- Selecting schools ...')
quipper_video_philippines_url = 'https://link.quipper.com/en/organizations/56c2b9f7c35ce42e180026da/dashboard'

quipper_school_philippines_url = 'https://link.quipper.com/en/organizations/52f4a97bbb8473000f000064/dashboard'

quipper_university_url = 'https://link.quipper.com/organizations/5aea684049f1fe0029000015/dashboard'

quipper_school_philippines_2017_url = 'https://link.quipper.com/en/organizations/58b75e136e392c6f6700000a/dashboard'

school_ls = [
    quipper_video_philippines_url, 
    quipper_school_philippines_url, 
    quipper_university_url, 
    quipper_school_philippines_2017_url
]

school_name = [
    'Quipper Video Philippines',
    'Quipper School Philippines',
    'Quipper University',
    'Quipper School Philippines 2017'
]

message = '''Please enter the number to select a school:
        1. Quipper Video Philippines
        2. Quipper School Philippines
        3. Quipper University
        4. Quipper School Philippines 2017
'''
selected_school_number = int(input(message))
selected_school_url = school_ls[selected_school_number-1]
selected_school_name = school_name[selected_school_number-1]
print(f'- Selected "{selected_school_name}"')
driver.get(selected_school_url)

# Disble pop up
pop_up_xpath = "//button[@class='btn btn-outline-primary']"
try:
    wait.until(EC.presence_of_element_located((By.XPATH, pop_up_xpath)))
except:
    sleep(10)

try:
    print('- Pop up ...')
    pop_up_close = driver.find_element_by_xpath(pop_up_xpath)
    driver.execute_script("arguments[0].click();", pop_up_close)
    print('- Close pop up. Ready to go!')
except:
    print('- No pop up. Ready to go!')

print('- Switching tab to Curriculum & Assignment ...')
curriculum_tab = selected_school_url.replace('dashboard', 'curriculum#curriculum')
driver.get(curriculum_tab)

# select category
print('- Selecting category ...')
select_category_xpath = '//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/div/div/div'
try:
    wait.until(EC.presence_of_element_located((By.XPATH, select_category_xpath)))
except:
    sleep(10)
select_category = driver.find_element_by_xpath(select_category_xpath)
select_category.click()
sleep(5)

category = "My school's courses"
print(f'- Selecting "{category}" ...')
my_school_course_xpath = f'//*[contains(text(), "{category}")]' # '//*[@id="react-select-3-option-6"] '
my_school_course = driver.find_element_by_xpath(my_school_course_xpath)
actions = ActionChains(driver)
actions.move_to_element(my_school_course).perform()
try:
    driver.execute_script("arguments[0].click();", my_school_course)
except:
    print('- Cannot select My School Courses')

print('- Searching keyword ...')

- Selecting schools ...
- Selected "Quipper School Philippines 2017"
- Pop up ...
- Close pop up. Ready to go!
- Switching tab to Curriculum & Assignment ...
- Selecting category ...
- Selecting "My school's courses" ...
- Searching keyword ...


# Task 2: Get the Filter Criteria from Sheet (subject, grade, module, topics)

In [196]:
# Task 2.1: fetch data from google sheets
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('gspread_credential.json',scope)
gc = gspread.authorize(creds)

In [197]:
subject_code = {
    'RW':'Reading and Writing Skills',
    'E':'English',
    'LT':'21st century literature'
}

school_grade = {
    'Quipper Video Philippines': [10,11,12],
    'Quipper School Philippines': [],
    'Quipper University': [],
    'Quipper School Philippines 2017': [1,2,3,4,5,6,7,8,9]
}

In [198]:
url_filter = 'https://docs.google.com/spreadsheets/d/1YvmsBmfjYxYi-aYJE2EX0566vyhxqMTwdir8zUtK-ls/edit#gid=2030777624'
sh = gc.open_by_url(url_filter)
sheet_name = 'Grammarly and Photo Check'
worksheet = sh.worksheet(sheet_name)
codes = worksheet.col_values(1)
grades = worksheet.col_values(2)
modules = worksheet.col_values(8)
topics = worksheet.col_values(7)

subjects = []
for code in codes:
    for abb, sub in subject_code.items():
        if abb.strip() == code.strip():
            subjects.append(sub)

codes = list(filter(None, codes))[1:] #1
grades = list(filter(None, grades))[1:] #2
modules = list(filter(None, modules))[1:] #3
topics = list(filter(None, topics))[1:] #3
subjects = list(filter(None, subjects)) #2

In [199]:
keywords = []
for i in range(len(subjects)):
    keyword = f'{subjects[i]} {grades[i]}' 
    keywords.append(keyword)

set_keywords = set(keywords)
# print(set_keywords)

content_filter = {}
for key in set_keywords:
    content_filter[key] = {}
# print(content_filter)

modules_grades = [[modules[i].strip(), keywords[i].strip()] for i in range(len(modules))]
# print(modules_grades)
set_modules_grades = []
for i in modules_grades:
    if i not in set_modules_grades:
        set_modules_grades.append(i)
# print(set_modules_grades)

for key, value in content_filter.items():
    for i in set_modules_grades:
        if key.strip() == i[-1].strip():
            value[i[0]] = []

modules_topics_ls = [[topics[i], modules[i]] for i in range(len(topics))]
modules_topics_dic = {}
for i in set_modules_grades:
    modules_topics_dic[i[0]] = []

for key, value in modules_topics_dic.items():
    for i in modules_topics_ls:
        if key.strip() == i[-1].strip():
            value.append(i[0].strip())
# print(modules_topics_dic)

for item in content_filter.values():
    for key1, value1 in item.items():
        # print(value1)
        for key2, value2 in modules_topics_dic.items():
            # print(value2)
            if key1 == key2:
                item[key1] = value2

# print(content_filter)

In [200]:
# filter if Quipper School Ph 2017 then only get certain courses of grade < 10
content_filter_school = {}
selected_grade = school_grade[selected_school_name]
for sch_subject, sch_module in content_filter.items():
    sch_grade = int(sch_subject[sch_subject.rfind(' ') + 1:])
    if sch_grade in selected_grade:
        content_filter_school[sch_subject] = sch_module

print('- Preparing these modules & topics ...\n')
for grade, content_modules in content_filter_school.items():
    for module, topics in content_modules.items():
        for topic in topics:
            name = f'{grade}: {module} - {topic}'
            print(f'     {name}')

- Preparing these modules & topics ...

     English 8: Cohesion in Writing - Misplaced Modifiers
     English 8: Cohesion in Writing - Dangling Modifiers
     English 8: Cohesion in Writing - Parallel Structure of Words and Phrases
     English 8: Cohesion in Writing - Parallel Structure of Clauses
     English 4: Understanding Words - Words with Prefixes Un-, In-, Im-, Dis-, Mis-, and Re-
     English 4: Understanding Words - Words with Prefixes Un-, In-, Im-, Dis-, Mis-, and Re-
     English 4: Understanding Words - Using a Glossary
     English 9: Fallacies - What Is a Logical Fallacy?
     English 9: Fallacies - Fallacies of Logos
     English 9: Fallacies - Fallacies of Pathos
     English 9: Fallacies - Fallacies of Ethos
     English 9: Monologue - What Is a Monologue?
     English 9: Monologue - Analyzing a Character For a Monologue
     English 9: Monologue - Writing a Monologue
     English 9: Making Predictions - Making Inferences vs. Making Predictions
     English 9: Maki

In [201]:
# # Task 1: Filter

# content_filter_Q_2017_daph = {
#     'English 8':{
#         'Cohesion in Writing': ['Dangling Modifiers', 'Parallel Structure of Words and Phrases', 'Parallel Structure of Clauses']
#     },
#     'English 9':{
#         'Fallacies': ['What Is a Logical Fallacy?', 'Fallacies of Logos', 'Fallacies of Pathos', 'Fallacies of Ethos'],
#         'Monologue': ['What Is a Monologue?', 'Analyzing a Character for a Monologue', 'Writing a Monologue'],
#         'Making Predictions': ['Making Inferences vs. Making Predictions', 'What Is Foreshadowing?', 'Analyzing a Foreshadowing']
#     }
# }

# content_filter_Q_video = {
#     '21st Century Literature from the Philippines and the World':{
#         'Asian Literature':['Indian Literature', 'Thai and Indonesian Literature'],
#         'African Literature':['Introduction to African Literature', 'African Prose', 'African Poetry', 'Modern African Literature'],
#     },
#     'Reading and Writing Skills':{
#         'Text as a Connected Discourse': ['Definition and Purposes of a Discourse', 'Types of Discourse', 'Literary Discourse'],
#     }
# }

# # content_filter_Q_video = {
# #     'Reading and Writing Skills':{
# #         'Text as a Connected Discourse': ['Definition and Purposes of a Discourse', 'Types of Discourse', 'Literary Discourse'],
# #     }
# # }

# content_filter_Q_2017_vy = {
#     'English 4':{
#         'Understanding Words': ['Words with Prefixes Un-, In-, Im-, Dis-, Mis-, and Re-', 'Words with Suffixes -Ful -Less, -Er, -Or, -Ible, and -Able', 'Using a Glossary']
#     },
#     'English 8':{
#         'Cohesion in Writing': ['Misplaced Modifiers'],
#     },
#     'English 9':{
#         'Monologue': ['Analyzing a Character for a Monologue'],
#     }
# }

# content_all = {
    
#     'English 4':{
#         'Understanding Words': ['Words with Prefixes Un-, In-, Im-, Dis-, Mis-, and Re-', 'Words with Suffixes -Ful -Less, -Er, -Or, -Ible, and -Able', 'Using a Glossary']
#     },
    
#     'English 8':{
#         'Cohesion in Writing': ['Misplaced Modifiers', 'Dangling Modifiers', 'Parallel Structure of Words and Phrases', 'Parallel Structure of Clauses']
#     },
    
#     'English 9':{
#         'Fallacies': ['What Is a Logical Fallacy?', 'Fallacies of Logos', 'Fallacies of Pathos', 'Fallacies of Ethos'],
#         'Monologue': ['What Is a Monologue?', 'Analyzing a Character for a Monologue', 'Writing a Monologue'],
#         'Making Predictions': ['Making Inferences vs. Making Predictions', 'What Is Foreshadowing?', 'Analyzing a Foreshadowing']
#     }
# }     

# Task 3: Scrape Content based on the Filter

In [202]:
# Task 2: Get the content  

# Task 2.1: Load content from QLink
def get_QLink_content(page_no):
    print(f'[PAGE] {page_no}: clicked')
    page = driver.find_element_by_xpath(f'//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/a[{page_no}]')
    # page.click()
    driver.execute_script("arguments[0].click();", page)
    sleep(3)
    page_source = BeautifulSoup(driver.page_source)
    sleep(2)
    page_content = page_source.find('div', class_='question-content-lesson-chapter Chapter--12HKG').get_text(strip=True)
    print(page_content, '\n\n')
    return page_content

# Task 2.2: Load PDF content
def get_PDF_content(module):
    folder_path = '/Users/admin/Environments/quipper_audit/'
    pdf_folder = os.listdir(folder_path)
    for pdf in pdf_folder:
        if module in pdf:
            raw = parser.from_file(folder_path + pdf)
            pdf_content = raw['content']
            print(len(pdf_content))
            # print(raw['content'])
            file1=open(f'/Users/admin/Environments/quipper_audit/{module}.txt',"a")
            file1.writelines(pdf_content)
            return pdf_content    

## To CSV

In [203]:
# grade_xpath = {
#     'english 4': 'a[1]',
#     'english 8': 'a[1]',
#     'english 9': 'a[1]'
# }

# # grade_xpath_Qvideo = {
# #     '21st Century Literature from the Philippines and the World': 'div/a'
# #     'Reading and Writing Skills': 'a[1]'
# # }

# grade_xpath_Qvideo = {
#     'Reading and Writing Skills': 'div[1]/a'
# }

# shared_xpath = '/html/body/section/div[1]/div[3]/div/div/div[2]/div/div[1]/div/div[2]/div[2]/div/div[1]/ul/'

# # input_grade = 'english 8'

# driver.execute_script("window.open('');")
# driver.switch_to.window(driver.window_handles[0])

# with open(f'content_scape_RW11.csv', 'w', newline = '') as file_output:
#     headers = ['grade', 'module', 'topic', 'page', 'content', 'code']
#     writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
#     writer.writeheader() 

#     # for grade, xpath in grade_xpath_Qvideo.items():

#     # we are now at the the course, module page
#     for grade, content_modules in content_filter_Q_video.items():
        
#         try:
#             # locate the course name box search
#             input_grade = grade
#             print(f'\n[GRADE] {input_grade}: clicked')
#             course_name_box = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/div/input')
#             course_name_box.clear()
#             course_name_box.send_keys(input_grade)
#             sleep(3)

#         except:
#             print('ERROR GRADE!!!')

#         # click search
#         search_button = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/button')
#         driver.execute_script("arguments[0].click();", search_button)
#         sleep(3)
        
#         if grade == input_grade:
#             for module, topics in content_modules.items():
                
#                 grade_button = driver.find_element_by_xpath(f"//*[contains(text(), '{grade}')]")
#                 driver.execute_script("arguments[0].click();", grade_button)
#                 sleep(3)

#                 # click module button
#                 print(f'\n[MODULE] {module}: clicked')
#                 module_button = driver.find_element_by_xpath(f"//*[contains(text(), '{module}')]")
#                 driver.execute_script("arguments[0].click();", module_button)
#                 sleep(3)

# #                     search = 'Some Text'
# #                     lc = search.lower()
# #                     uc = search.upper()
# #                     xpath = f"//p[contains(translate(text(), '{lc}', '{uc}'), '{uc}')]"

#                 for topic in topics:
#                     # click topic button

#                     try:
#                         print(f'\n[TOPIC] {topic}: clicked')
#                         topic_button = driver.find_element_by_xpath(f"//*[contains(text(), '{topic}')]")
#                         driver.execute_script("arguments[0].click();", topic_button)
#                         sleep(3)

#                     except:
#                         print('ERROR TOPIC!!!')

#                     # get the latest page source
#                     page_source = BeautifulSoup(driver.page_source)
#                     all_page = page_source.find('div', class_ = 'stepper')
#                     pages = all_page.find_all('a', class_='stepper-step')

#                     for page_no in range(1, len(pages) + 1):
#                         page_title = f'page_{page_no}'
#                         page_code = f'START###{input_grade} - {module} - {topic} - {page_title}###END'.upper()
#                         page_content = page_code + get_QLink_content(page_no)

#                         writer.writerow({
#                             headers[0]: input_grade,
#                             headers[1]: module,
#                             headers[2]: topic,
#                             headers[3]: page_title,
#                             headers[4]: page_content,
#                             headers[5]: page_code
#                         })

#                     try:
#                         pdf_code = f'START###{input_grade} - {module} - {topic} - Study Guide###END'.upper()
#                         pdf_content = pdf_code + get_PDF_content(module)

#                     except:
#                         pdf_content = 'NO STUDY GUIDE'

#                     writer.writerow({
#                         headers[0]: input_grade,
#                         headers[1]: module,
#                         headers[2]: topic,                          
#                         headers[3]:'Study Guide',
#                         headers[4]: pdf_content,
#                         headers[5]: pdf_code
#                     })  

# print('Mission Completed!')

## To GSheet

In [204]:
row = []
# we are now at the the course, module page
for grade, content_modules in content_filter_school.items():

    grade_backup = grade[:grade.rfind(' ')]

    # handle 21st century course 
    if grade == '21st century literature 11':
        try:
            # locate the course name box search
            print(f'\n[GRADE] {grade}: clicked')
            course_name_box = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/div/input')
            course_name_box.clear()
            # course_name_box.send_keys(Keys.RETURN)
            sleep(3)

        except:
            print('ERROR GRADE!!!')

        # click search
        search_button = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/button')
        driver.execute_script("arguments[0].click();", search_button)
        sleep(3)

    elif grade == 'Reading and Writing Skills 11':
        try:
            # locate the course name box search
            print(f'\n[GRADE] {grade}: clicked')
            course_name_box = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/div/input')
            course_name_box.clear()
            course_name_box.send_keys(grade_backup)
            sleep(3)

        except:
            print('ERROR GRADE!!!')
        # click search
        search_button = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/button')
        driver.execute_script("arguments[0].click();", search_button)
        sleep(3)

    else:
        try:
            # locate the course name box search
            print(f'\n[GRADE] {grade}: clicked')
            course_name_box = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/div/input')
            course_name_box.clear()
            course_name_box.send_keys(grade)
            sleep(3)

        except:
            print('ERROR GRADE!!!')

        # click search
        search_button = driver.find_element_by_xpath('//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div[3]/div/button')
        driver.execute_script("arguments[0].click();", search_button)
        sleep(3)

    for module, topics in content_modules.items():

        try:
            grade_button = driver.find_element_by_xpath(f"//*[contains(text(), '{grade}')]")
            driver.execute_script("arguments[0].click();", grade_button)
            sleep(3)

        except:
            grade_button = driver.find_element_by_xpath(f"//*[contains(text(), '{grade_backup}')]")
            driver.execute_script("arguments[0].click();", grade_button)
            sleep(3)

        # click module button
        print(f'\n[MODULE] {module}: clicked')
        module_button = driver.find_element_by_xpath(f"//*[contains(text(), '{module}')]")
        driver.execute_script("arguments[0].click();", module_button)
        sleep(3)

#                     search = 'Some Text'
#                     lc = search.lower()
#                     uc = search.upper()
#                     xpath = f"//p[contains(translate(text(), '{lc}', '{uc}'), '{uc}')]"

        for topic in topics:
            # click topic button

            try:
                print(f'\n[TOPIC] {topic}: clicked')
                topic_button = driver.find_element_by_xpath(f"//*[contains(text(), '{topic}')]")
                driver.execute_script("arguments[0].click();", topic_button)
                sleep(3)

            except:
                print('ERROR TOPIC!!! Unable to click')

            try:
                # get the latest page source
                page_source = BeautifulSoup(driver.page_source)
                all_page = page_source.find('div', class_ = 'stepper')
                pages = all_page.find_all('a', class_='stepper-step')

                for page_no in range(1, len(pages) + 1):
                    page_title = f'page_{page_no}'
                    page_code = f'{grade} - {module} - {topic} - {page_title}'.upper()
                    page_content = f'{page_code} {get_QLink_content(page_no)}'

                    row.append([grade, module, topic, page_title, page_content, page_code])

                try:
                    pdf_code = f'{grade} - {module} - {topic} - Study Guide'.upper()
                    pdf_content = pdf_code + get_PDF_content(module)

                except:
                    pdf_content = 'NO STUDY GUIDE'                  

                row.append([grade, module, topic, 'Study Guide', pdf_content, pdf_code])
            
            except AttributeError:
                row.append([grade, module, topic, 'NA', 'NA', 'NA'])

            # print(f'ROW: {row}')


print('Mission Completed!')

ramatic monologues?Report errors 


[PAGE] 2: clicked
Learn about It!Amonologueis a speech that presents the words and innermost thoughts of a single author. Amonologistis a person who delivers a monologue. Monologues present ideas, thoughts, and motives by a speaker through verbal and non-verbal elements such as facial expressions and gestures.Report errors 


[PAGE] 3: clicked
Learn about It!Types of MonologueSoliloquyA** soliloquy** is a monologue that shows the character talking loudly to himself at length. In a soliloquy, the audience and at times the figures present in that particular scene can hear what the character says, but he is unaware of those who can overhear him.An example of a soliloquy is the Balcony scene in Shakespeare’s “Romeo and Juliet”:“O Romeo, Romeo! Wherefore art thou Romeo?Deny thy father and refuse thy name;Or, if thou wilt not, be but sworn my love,And I’ll no longer be a Capulet.”Dramatic MonologueAdramatic monologueis a formal or informal speech that the 

## Task 4: Df to Sheet

In [205]:
df_content = pd.DataFrame(np.array(row), columns=['grade', 'module', 'topic', 'page', 'content', 'code'])
df_content

,grade,module,topic,page,content,code
0,English 8,Cohesion in Writing,Misplaced Modifiers,page_1,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...
1,English 8,Cohesion in Writing,Misplaced Modifiers,page_2,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...
2,English 8,Cohesion in Writing,Misplaced Modifiers,page_3,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...
3,English 8,Cohesion in Writing,Misplaced Modifiers,page_4,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...
4,English 8,Cohesion in Writing,Misplaced Modifiers,page_5,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...,ENGLISH 8 - COHESION IN WRITING - MISPLACED MO...
...,...,...,...,...,...,...
135,English 9,Making Predictions,Analyzing a Foreshadowing,page_3,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...
136,English 9,Making Predictions,Analyzing a Foreshadowing,page_4,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...
137,English 9,Making Predictions,Analyzing a Foreshadowing,page_5,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...
138,English 9,Making Predictions,Analyzing a Foreshadowing,page_6,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...,ENGLISH 9 - MAKING PREDICTIONS - ANALYZING A F...


In [206]:
def ToSheet(url, df, sheet_title):
    sh = gc.open_by_url(url)
    try:
        worksheet_output = sh.add_worksheet(title=sheet_title, rows=f"10000", cols="50")
        ws = sh.worksheet(sheet_title)
    except:
        ws = sh.worksheet(sheet_title)
        ws.clear()
    # ws = sh.worksheet(sheet_title)
    gd.set_with_dataframe(ws, df)

In [207]:
url = 'https://docs.google.com/spreadsheets/d/1vpnmWIIRlKOGttpPb1iJOtXwS-k4crCd98iZMFRgUQ0/edit?usp=sharing'

In [208]:
ToSheet(url, df_content, 'QLink Content')